In [95]:
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [96]:
filename = "articles_info.csv" # имя файла, в который будем сохранять результат
driver_path = "D:\IDE_SF\chromedriver.exe" # укажите ваш путь к chromedriver, который вы загрузили ранее
base_dir = r"D:\IDE_SF\Учебные проекты\13. Парсинг\data" # укажите директорию, в которую будем сохранять файл
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.7390.123 Safari/537.36" # ваш user-agent, узнать его можно тут: https://юзерагент.рф, смотреть через браузер Chrome
start_time = time() # время начала выполнения программы

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\User\AppData\Local\Temp\ipykernel_10940\2695948138.py:2: SyntaxWarning: invalid escape sequence '\I'
  driver_path = "D:\IDE_SF\chromedriver.exe" # укажите ваш путь к chromedriver, который вы загрузили ранее


In [97]:
# Время загрузки

def get_load_time(article_url, user_agent):
    #будем ждать 3 секунды, иначе выводить exception и присваивать константное значение
    try:
        # меняем значение заголовка. По умолчанию указано, что это python-код
        headers = {
            "User-Agent": user_agent
        }
        # делаем запрос по url статьи article_url
        response = requests.get(
            article_url, headers=headers, stream=True, timeout=3.000
        )
        # получаем время загрузки страницы
        load_time = response.elapsed.total_seconds()
    except Exception as e:
        print(e)
        load_time = ">3"
    return load_time

In [98]:
# Сохранение результатов в фаил

def write_to_file(output_list, filename, base_dir):
    base_path = Path(base_dir)
    base_path.mkdir(parents=True, exist_ok=True)
    file_path = base_path / filename

    fieldnames = ["id", "load_time", "rank", "points", "title", "url", "comments"]

    file_exists = file_path.exists()

    with open(file_path, "a", newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(
            csvfile,
            fieldnames=fieldnames,
            quoting=csv.QUOTE_ALL  # <-- ключевой момент: экранируем запятые в title
        )

        if not file_exists:
            writer.writeheader()

        for row in output_list:
            writer.writerow(row)

In [99]:
# Проверка доступности страницы

def connect_to_base(browser, page_number):
    base_url = "https://news.ycombinator.com/news?p={}".format(page_number)
    for connection_attempts in range(1,4): # совершаем 3 попытки подключения
        try:
            browser.get(base_url)
            # ожидаем пока элемент table с id = 'hnmain' будет загружен на страницу
            # затем функция вернет True иначе False 
            WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.ID, "hnmain"))
            )
            return True
        except Exception as e:
            print(e)
            print("Error connecting to {}.".format(base_url))
            print("Attempt #{}.".format(connection_attempts))
    return False

In [100]:
# Извлечение атрибутов

def parse_html(html, user_agent):
    soup = BeautifulSoup(html, "html.parser")
    output_list = []
   
    # ищем в объекте soup object id, rank, score и title статьи
    tr_blocks = soup.find_all("tr", class_="athing")
    article = 0
    for tr in tr_blocks:
        article_id = tr.get("id") # id
        article_url = tr.find_all("a")[1]["href"]

        # иногда статья располагается не на внешнем сайте, а на ycombinator
        # тогда article_url у нее не полный, а добавочный, с параметрами.
        # например item?id=200933. Для этих случаев будем добавлять url до полного
        if "item?id=" in article_url or "from?site=" in article_url:
            article_url = f"https://news.ycombinator.com/{article_url}"
        load_time = get_load_time(article_url, user_agent)
        # иногда рейтинга может не быть, поэтому воспользуемся try

        try:
            score = soup.find(id=f"score_{article_id}").string
        except Exception as e:
            print(e)
            score = "0 points"
            
    # ___________________________ЗАДАНИЕ - ДОБАВЛЕНИЕ КОЛИЧЕСТВА КОММЕНТАРИЕВ_____________________________________     
    
        comments = "0 comments"
        subtext = tr.find_next_sibling("tr").find("td", class_="subtext")
        if subtext:
            comment_link = subtext.find_all("a")[-1]
            if comment_link and "comment" in comment_link.text:
                comments = comment_link.text.strip()
                
    # ___________________________ЗАДАНИЕ - ДОБАВЛЕНИЕ КОЛИЧЕСТВА КОММЕНТАРИЕВ_____________________________________                 
        article_info = {
            "id": article_id,
            "load_time": load_time,
            "rank": tr.span.string,
            "points": score,
            "title": tr.find(class_="titleline").get_text(strip=True),
            "url": article_url,
            "comments": comments    #
        }

        # добавляем информацию о статье в список
        output_list.append(article_info)
        article += 1
    return output_list

In [101]:
start_time = time() # время начала выполнения программы

# инициализируем веб драйвер
browser = webdriver.Chrome(
    service = Service(driver_path)
)

# перебираем страницы и собираем нужную информацию
for page_number in range(10):
    print("getting page " + str(page_number) + "...")
    if connect_to_base(browser, page_number):
        sleep(5)
        output_list = parse_html(browser.page_source, user_agent)
        write_to_file(output_list, filename, base_dir)

    else:
        print("Error connecting to hacker news")

# завершаем работу драйвера
browser.close()
sleep(1)
browser.quit()
end_time = time()
elapsed_time = end_time - start_time
print("run time: {} seconds".format(elapsed_time))

getting page 0...
HTTPSConnectionPool(host='thezvi.substack.com', port=443): Max retries exceeded with url: /p/openai-moves-to-complete-potentially (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_PROTOCOL_VERSION] tlsv1 alert protocol version (_ssl.c:1000)')))
HTTPSConnectionPool(host='medium.com', port=443): Max retries exceeded with url: /@jonas.eschenburg/how-i-stopped-worrying-and-started-loving-the-assembly-4fd00e786c60 (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_PROTOCOL_VERSION] tlsv1 alert protocol version (_ssl.c:1000)')))
HTTPSConnectionPool(host='www.telegraph.co.uk', port=443): Max retries exceeded with url: /world-news/2025/10/28/sudan-bloodied-sands-massacre-thousands/ (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
getting page 1...
HTTPSConnectionPool(host='thezvi.substack.com', port=443): Max retries exceeded with url: /p/openai-moves-to-complete-potentially (Caused by SSLError(SS

In [104]:
import pandas as pd

articles_data = pd.read_csv(
    'data/articles_info.csv',
    names=["id", "load_time", "rank", "points", "title", "url", "Comments"],
    encoding='UTF-8',
    header=1
)

articles_data.head()

,id,load_time,rank,points,title,url,Comments
0,45783114,0.379227,2.0,82 points,Chat Control proposal fails again after public...,https://andreafortuna.org/2025/11/01/chat-cont...,28 comments
1,45782981,0.838614,3.0,81 points,GHC now runs in the browser(haskell.org),https://discourse.haskell.org/t/ghc-now-runs-i...,14 comments
2,45783640,0.243027,4.0,14 points,Show HN: Why write code if the LLM can just do...,https://github.com/samrolken/nokode,7 comments
3,45781298,0.260891,5.0,162 points,SQLite concurrency and why you should care abo...,https://jellyfin.org/posts/SQLite-locking/,71 comments
4,45783440,0.703645,6.0,14 points,How I Learned to Stop Worrying and Love My Shi...,https://www.thedriftmag.com/how-i-learned-to-s...,1 comment
